In [ ]:
!pip install pyspark
!apt install openjdk-8-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=b2a9ac0e22ba33ebb47501454cf0ce41907fb740edfa171c458cd38d537aa73b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
E: Unable to locate package openjdk-8-headless


In [ ]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as fn
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf

In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [5]:
wine_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/content/drive/MyDrive/3-2 university/Big_data_process_and_application/input/Wine.csv')
wine_df.show(5)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|
|  13.16|      2.36|2.67|        18.6|      101|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17|   1185|               1|
|  14.37|      1.95| 2

In [6]:
wine_df.drop_duplicates(['Customer_Segment']).count()

3

In [8]:
# dense vector로 데이터 변환
from pyspark.ml.feature import VectorAssembler

# Input all the features in one vector column
assembler = VectorAssembler(inputCols=['Alcohol',
                                       'Malic_Acid',
                                       'Ash',
                                       'Ash_Alcanity',
                                       'Magnesium',
                                       'Total_Phenols',
                                       'Flavanoids',
                                       'Nonflavanoid_Phenols',
                                       'Proanthocyanins',
                                       'Color_Intensity',
                                       'Hue',
                                       'OD280',
                                       'Proline'], outputCol = 'features')

wine_df = assembler.transform(wine_df)
wine_df.select('features').show(5, False) # False : 생략 없이 출력

+---------------------------------------------------------------------+
|features                                                             |
+---------------------------------------------------------------------+
|[14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]|
|[13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0] |
|[13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0] |
|[14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0] |
|[13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0] |
+---------------------------------------------------------------------+
only showing top 5 rows



In [9]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [10]:
# 3 가지 와인으로 군집
km = KMeans().setK(3)
kmModel = km.fit(wine_df)

In [11]:
# prediction 수행
predictions = kmModel.transform(wine_df)
predictions.show(5)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+----------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|            features|prediction|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+----------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|[14.23,1.71,2.43,...|         1|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|[13.2,1.78,2.14,1...|         1|
|  13.16|      2.36|2.67|

In [12]:
# 평가
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print('Silhouette with squared euclidean distance = ', str(silhouette))

Silhouette with squared euclidean distance =  0.7322991109041611
